In [1]:
from preprocess import *
import keras
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.models import load_model

Using TensorFlow backend.


In [2]:
from keras import backend as K
# 用GPU 
# K.tensorflow_backend._get_available_gpus()

In [3]:
save_path = "/home/ais3/Desktop/Linux/soundAnalysis/training_data/correct/cqt/np/"
def save_data_npy(filename):
    cur_file = img_path + filename + extension
    cqt_img = np.array(Image.open(cur_file).convert('L')) # Image.open(cur_file).convert('RGB')
    cqt_img = np.expand_dims(cqt_img, axis=0)
    #plt.imshow(cqt_img)
    np.save(save_path + filename, cqt_img)
    #print(cqt_img.shape)

### one-hot encoding
# Input: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
def get_labels(path=save_path):
    labels = os.listdir(path)
    label_indices = np.arange(1, len(labels))
    # to_categorical: one hot encoding
    return labels, label_indices, to_categorical(label_indices)

In [4]:
"""
# Load Image, and save to npy
from PIL import Image
extension = ".png"
save_path = "/home/ais3/Desktop/Linux/soundAnalysis/training_data/correct/cqt/np/"
img_path = "/home/ais3//Desktop/Linux/soundAnalysis/training_data/correct/cqt/"
for sec in range(1,30): # sec
    save_data_npy("1_"+str(sec))
    
# Load Image, and save to npy
from PIL import Image
extension = ".png"
save_path = "/home/ais3/Desktop/Linux/soundAnalysis/training_data/incorrect/cqt/np/"
img_path = "/home/ais3//Desktop/Linux/soundAnalysis/training_data/incorrect/cqt/"

for sec in range(1,30): # sec
    save_data_npy("2_"+str(sec))
    """

'\n# Load Image, and save to npy\nfrom PIL import Image\nextension = ".png"\nsave_path = "/home/ais3/Desktop/Linux/soundAnalysis/training_data/correct/cqt/np/"\nimg_path = "/home/ais3//Desktop/Linux/soundAnalysis/training_data/correct/cqt/"\nfor sec in range(1,30): # sec\n    save_data_npy("1_"+str(sec))\n    \n# Load Image, and save to npy\nfrom PIL import Image\nextension = ".png"\nsave_path = "/home/ais3/Desktop/Linux/soundAnalysis/training_data/incorrect/cqt/np/"\nimg_path = "/home/ais3//Desktop/Linux/soundAnalysis/training_data/incorrect/cqt/"\n\nfor sec in range(1,30): # sec\n    save_data_npy("2_"+str(sec))\n    '

In [5]:
dict_labels = {"incorrect":0, "correct":1}
num_data = 58

save_path = "/home/ais3/Desktop/Linux/soundAnalysis/training_data/correct/cqt/np/"
# Get available labels
labels, indices, _ = get_labels(save_path)

# Getting first arrays
X = np.load(save_path + labels[0])
y = np.zeros(X.shape[0])
y[0] = 1

In [6]:
# Append all of the dataset into one single array, same goes for y
for i, label in enumerate(labels[1:]):
    x = np.load(save_path + label)
    X = np.vstack((X, x))
    y = np.append(y, np.full(x.shape[0], fill_value= dict_labels["correct"]))
print(X.shape[0], len(y))
assert X.shape[0] == len(y)


29 29


In [7]:
x = np.load(save_path + labels[0])
x.shape

(1, 360, 480)

In [8]:
save_path = "/home/ais3/Desktop/Linux/soundAnalysis/training_data/incorrect/cqt/np/"
labels, indices, _ = get_labels(save_path)

for i, label in enumerate(labels):
    x = np.load(save_path + label)
    X = np.vstack((X, x))
    y = np.append(y, np.full(x.shape[0], fill_value= dict_labels["incorrect"]))
print(X.shape[0], len(y))
assert X.shape[0] == len(y)

58 58


In [9]:
# loading data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= (1 - 0.6), random_state=48, shuffle=True)
X_train = X_train.reshape(X_train.shape[0], 360, 480, 1)
X_test = X_test.reshape(X_test.shape[0], 360, 480, 1)

# one-hot encoding
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

In [10]:
y_test_hot

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [11]:
training_path = "/home/ais3/Desktop/Linux/soundAnalysis/training_data/cqt/np/"
correct_path = "/home/ais3/Desktop/Linux/soundAnalysis/training_data/correct/cqt/np/"
incorrect_path = "/home/ais3/Desktop/Linux/soundAnalysis/training_data/incorrect/cqt/np"



In [ ]:
# create Sequential model
model = Sequential()
model.add(Conv2D(58, kernel_size=(2, 2), input_shape = (360,480,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# Flatten
model.add(Flatten())
# FNN layer: 128 output
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
# Add output layer
model.add(Dense(2, activation='softmax'))
# Compile: choose loss function、optimize method
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
# training, 訓練過程會存在 train_history
model.fit(X_train, y_train_hot, batch_size=100, epochs=200, verbose=1, validation_data=(X_test, y_test_hot))
# 顯示損失函數、訓練成果(分數)
score = model.evaluate(X_test, y_test_hot, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 34 samples, validate on 24 samples
Epoch 1/200
